 TODO: 
 
 - Step 1: Download data from ORCESTRA
 - Step 2: MultiAssayExperiment + BumpyMatrix deconstructor

In [ ]:
from workflow.scripts.python.data.helpers import *
import yaml
import itertools

In [2]:
# Global variables (?)
RAW_DATA_PATH = f"../../../rawdata/"
PROC_DATA_PATH = f"../../../procdata/"
RESULTS_DATA_PATH = f"../../../results/"

# Step 1: Set the dataset name for the pipeline

In [3]:
# Set dataset name
# Should be in format "RADCURE" or "Head-Neck-Radiomics-HN1"
# DATASET_NAME = "Head-Neck-Radiomics-HN1"
# DATASET_NAME = "HNSCC"
DATASET_NAME = "RADCURE"

# Step X: Load the config file for this dataset

In [4]:
config_file = f"../../config/{DATASET_NAME}.yaml"
config = yaml.safe_load(open(config_file))

# Step X: Pick imaging feature signatures to use
This can be blank, or you can list the signatures from workflow/signatures/ that you want to use. The string must match the filename of the signature file exactly.

In [5]:
# SIGNATURES = ['all_features']

# Step X: Make output directories for this pipeline

In [5]:
# Make clinical procdata folder
path_to_proc_clinical = os.path.join(PROC_DATA_PATH, DATASET_NAME, 'clinical')
if not os.path.exists(path_to_proc_clinical):
    os.makedirs(path_to_proc_clinical)

# Make feature procdata folders
data_sources = ['radiomics', 'deep_learning']
data_types = ['clinical', 'features']

for combo in itertools.product([DATASET_NAME], data_sources, data_types):
    path_to_create = os.path.join(PROC_DATA_PATH, *combo)
    
    if not os.path.exists(path_to_create):
        os.makedirs(path_to_create)

# Optionally make train and test output folders
if config["train_test_split"]["split"] is True:
    #### MAKE TRAIN AND TEST OUTPUT DIRECTORIES ####
    split_data_types = ['clinical', 'train_features', 'test_features']
    for combo in itertools.product([DATASET_NAME], data_sources, ['train_test_split'], split_data_types):
        path_to_create = os.path.join(PROC_DATA_PATH, *combo)
        
        if not os.path.exists(path_to_create):
            os.makedirs(path_to_create)

In [6]:
# Results data folder creation
path_to_results_dataset = os.path.join(RESULTS_DATA_PATH, DATASET_NAME)

if not os.path.exists(path_to_create):
    os.makedirs(path_to_create)

# Step X: Load clinical data

In [7]:
clinical_data_path = os.path.join(RAW_DATA_PATH, DATASET_NAME, f"clinical/{DATASET_NAME}.csv")

# Load clinical data into a pandas dataframe
clinical_data = loadFileToDataFrame(clinical_data_path)
print(f"Clinical data loaded with {len(clinical_data)} patients.")

Clinical data loaded with 3346 patients.


# Step X: Clean clinical data
- Remove any specified exceptions in config file
- Set up the outcome variable column by making sure it is a boolean

In [8]:
# Get exclusion variable dictionary from config file
exclusion_clinical_variables = config["exclusion_variables"]

if exclusion_clinical_variables:
    print(f"Will exclude clinical variables: {exclusion_clinical_variables.keys()}")
    # Drop rows with values in the exclusion variables
    clinical_data = subsetDataframe(clinical_data, excludeDict=exclusion_clinical_variables)
    print("Clinical data updated, now has", len(clinical_data), "patients.")
else:
    print("No exclusion variables found in config file.")

Will exclude clinical variables: dict_keys(['Ds Site'])
Clinical data updated, now has 3118 patients.


### Outcome Variable setup
This data will be used for a Cox Proportional Hazards model, which expects time and event outcome variables. Time must be a continuous variable, and event must be a binary variable.
Event Variable must be in the format where 1 is the event (e.g. death), and 0 is non-event (e.g. alive).
In this pipeline, we are expecting the time to event to be in years, so will convert any other units to years.

In [9]:
# Set up ouptput variable columns for modelling
time_column_label = config["outcome_variables"]["time_label"]
event_column_label = config["outcome_variables"]["event_label"] 

# def survivalTimeColumnSetup
# Confirm that time column is numeric
if not np.issubdtype(clinical_data[time_column_label].dtype, np.number):
    raise ValueError(f"Time column {time_column_label} is not numeric. Please confirm time label in {DATASET_NAME}.yaml is the correct column or convert to numeric.")
else:
    print(f"Time column {time_column_label} is numeric. Making copy with standardized column name.")
    if config["outcome_variables"]["convert_to_years"]:
        print(f"Converting time column {time_column_label} from days to years.")
        clinical_data["survival_time_in_years"] = clinical_data[time_column_label] / 365
    else:
        clinical_data["survival_time_in_years"] = clinical_data[time_column_label]


# def survivalEventColumnSetup
# Determine what type of value is in event column
event_variable_type = type(clinical_data[event_column_label][0])
if np.issubdtype(event_variable_type, np.number):
    print(f"Event column {event_column_label} is binary. Making copy with standardized column name.")
    clinical_data["survival_event_binary"] = clinical_data[event_column_label]

elif np.issubdtype(event_variable_type, np.bool_):
    print(f"Event column {event_column_label} is boolean. Converting to binary.")
    clinical_data["survival_event_binary"] = clinical_data[event_column_label].astype(int)

elif np.issubdtype(event_variable_type, np.str_):
    print(f"Event column {event_column_label} is string. Checking what values are present in the column.")

    event_column_values = clinical_data[event_column_label].str.lower().unique()

    if len(event_column_values) != 2:
        raise ValueError(f"Event column {event_column_label} can only have two values. Please confirm event label in {DATASET_NAME}.yaml is the correct column or update to have only two values.")
    
    # Check if alive and dead are present in the event column
    if 'alive' in event_column_values and 'dead' in event_column_values: 
        print(f"Converting to binary where 0 is alive and 1 is dead.")

        clinical_data['survival_event_binary'] = clinical_data[event_column_label].str.lower().replace({'alive': '0', 'dead': '1'}).astype(int)

    else:
        raise ValueError(f"Event column {event_column_label} doesn't contain any variation of 'alive' and 'dead'. Please confirm event label in {DATASET_NAME}.yaml is the correct column.")

Time column Length FU is numeric. Making copy with standardized column name.
Event column Status is string. Checking what values are present in the column.
Converting to binary where 0 is alive and 1 is dead.


### Set patient ID as index

In [10]:
clinical_patient_identifier = getPatientIdentifierLabel(clinical_data)
clinical_data.set_index(clinical_patient_identifier, inplace=True)

## Save out cleaned and filtered clinical data

In [22]:
clinical_data.to_csv(os.path.join(PROC_DATA_PATH,DATASET_NAME,f"clinical/cleaned_filtered_clinical_{DATASET_NAME}.csv"))

# Step X: Get list of image types in each image feature folder
Will be looping over each of these for processing

In [11]:
readii_features_dir = os.path.join(RAW_DATA_PATH, DATASET_NAME, f"readii_outputs")
fmcib_features_dir = os.path.join(RAW_DATA_PATH, DATASET_NAME, f"fmcib_outputs")

In [12]:
# Get list of image types in the radiomic and fmcib feature directories
radiomic_image_types = sorted([file.removeprefix('radiomicfeatures_').removesuffix("_" + DATASET_NAME + ".csv") 
                        for file in os.listdir(readii_features_dir)])

fmcib_image_types = sorted([file.removeprefix('fmcibfeatures_').removesuffix("_" + DATASET_NAME + ".csv") 
                           for file in os.listdir(fmcib_features_dir)])

# For each image type:

In [13]:
radiomics_procdata_path = os.path.join(PROC_DATA_PATH, DATASET_NAME, "radiomics")

for image_type in radiomic_image_types:
    print(f"Processing radiomic features for {image_type}")
    # Load the feature data
    readii_output_data = loadFileToDataFrame(f"{readii_features_dir}/radiomicfeatures_{image_type}_{DATASET_NAME}.csv")

    print(f"Radiomic data loaded with {len(readii_output_data)} patients.")

    # Set the patient ID as the index
    radiomic_patient_identifier = getPatientIdentifierLabel(readii_output_data)

    # Check data for duplicated rows and remove the second copy
    readii_output_data = readii_output_data.drop_duplicates(subset=[radiomic_patient_identifier, "series_UID","image_modality","seg_modality","seg_ref_image", "roi"])

    readii_output_data.set_index(radiomic_patient_identifier, inplace=True)

    ######### FIND COMMON PATIENTS #########
    # Filter the clinical and image features to only include patients with imaging and clinical data based on image features index
    # e.g. patients with only clinical data will not be included
    common_patient_index = readii_output_data.index.intersection(clinical_data.index)
    # Select just the common patients from clinical and image feature data
    common_clinical_data = clinical_data.loc[common_patient_index]
    common_readii_output_data = readii_output_data.loc[common_patient_index]
    print(f"Patients with both clinical and radiomic features: {len(common_clinical_data)}")
    print(f"Number of segmentations with radiomic features: {len(common_readii_output_data)}")

    # Get just the radiomic feature columns from the dataframe, remove any metadata/diagnostics columns
    radiomic_feats_only = getOnlyPyradiomicsFeatures(common_readii_output_data)
    print(f"Number of radiomic features: {(radiomic_feats_only.shape[1])}")

    ######### GET SURVIVAL OUTCOME LABELS #########
    # Get the survival time and event columns
    survival_labels = common_clinical_data[["survival_time_in_years", "survival_event_binary"]]
    surv_labelled_radiomic_feats = survival_labels.join(radiomic_feats_only)

    # Save outputs at this point
    common_clinical_data.to_csv(os.path.join(radiomics_procdata_path, f"clinical/merged_clinical_{DATASET_NAME}.csv"))
    common_readii_output_data.to_csv(os.path.join(radiomics_procdata_path, f"features/merged_radiomicfeatures_{image_type}_{DATASET_NAME}.csv"))
    # Save out labelled radiomic feature data
    surv_labelled_radiomic_feats.to_csv(os.path.join(radiomics_procdata_path, f"features/labelled_radiomicfeatures_only_{image_type}_{DATASET_NAME}.csv"))


    if config["train_test_split"]["split"] is True:
        print("Splitting clinical and radiomic features only data into training and test sets.")
        # Split the data into training and test sets
        split_variable = config["train_test_split"]["split_variable"]

        # Check for rows of clinical dataframe that don't have a one of the values in the split variable dictionary
        splitClinical, splitFeatures = splitDataSetup(common_clinical_data, surv_labelled_radiomic_feats, 
                                                        splitVariables = config["train_test_split"]["split_variable"], 
                                                        imputeValue = config["train_test_split"]["impute"])
        
        # Set up train test output path
        train_test_dir = os.path.join(radiomics_procdata_path, "train_test_split")

        # Save out training and test clinical data
        splitClinical['training'].to_csv(os.path.join(train_test_dir, f"clinical/train_merged_clinical_{DATASET_NAME}.csv"))
        splitClinical['test'].to_csv(os.path.join(train_test_dir, f"clinical/test_merged_clinical_{DATASET_NAME}.csv"))

        # Save out training and test radiomic feature data
        splitFeatures['training'].to_csv(os.path.join(train_test_dir, f"train_features/train_labelled_radiomicfeatures_only_{image_type}_{DATASET_NAME}.csv"))
        splitFeatures['test'].to_csv(os.path.join(train_test_dir, f"test_features/test_labelled_radiomicfeatures_only_{image_type}_{DATASET_NAME}.csv"))

        print(f"{len(splitClinical['training'])} training patients and {len(splitClinical['test'])} test patients.")
        print(f"{len(splitFeatures['training'])} training GTVs and {len(splitFeatures['test'])} test GTVs.")
        print("------------------------------------------------------------")
        print()

    else:
        print("No train test split specified in config file.")


Processing radiomic features for original
Radiomic data loaded with 5988 patients.
Multiple patient identifier labels found. Using patient_ID.
Patients with both clinical and radiomic features: 2920
Number of segmentations with radiomic features: 2920
Number of radiomic features: 1317
Splitting clinical and radiomic features only data into training and test sets.
Made copy of split variable with imputed columns: RADCURE-challenge_imputed
Getting split for  RADCURE-challenge_imputed
2207 training patients and 713 test patients.
2207 training GTVs and 713 test GTVs.
------------------------------------------------------------

Processing radiomic features for randomized_sampled_full
Radiomic data loaded with 5988 patients.
Multiple patient identifier labels found. Using patient_ID.
Patients with both clinical and radiomic features: 2920
Number of segmentations with radiomic features: 2920
Number of radiomic features: 1317
Splitting clinical and radiomic features only data into training a

# Scratch code

In [ ]:
# import os
# os.getcwd()